# Notebook that is used for preprocessing the data that belongs to FollowTheMoney. Part of the code in this notebook was taken from https://github.com/asreview-ftm-hackathon/Data

# Package Imports

In [20]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd 
from bs4 import BeautifulSoup
import string
import re
import datetime
import numpy as np
import warnings



# Import Data

In [2]:
process = pd.read_csv('https://github.com/ftmnl/asr/raw/main/data/allExport.csv', sep='|')     # Used for the first part of the dataset that is on Github

In [14]:
process_new = pd.read_csv(r'C:\Users\MichaG\Documents\Scriptie\Data-main\export_dataset2.csv') # Used for the second part of the dataset that is not on
                                                                                               # Github

In [3]:
#remove noise
process = process.dropna()

process = process.rename(columns = {"file_name_sort": "title", "content": "abstract"})

remove = ['.DS_Store', 'NaN', 'Readme.md']
process = process[~process.title.isin(remove)]

# Transform Content

In [4]:
#abstract html to string

translate_table = dict((ord(char), None) for char in string.punctuation)

def prettify(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = text.replace("\r", "")
    text = text.replace('\n', '')
    #text = text.translate(translate_table)
    return str(text)

process.abstract = process.abstract.apply(prettify)

# Search formats

In [5]:
#extract id, type and date from title

import re

numberlist = []
typelist = []
datelist = []

for title in process.title:
    id = re.search('^[0-9\.]+', title)
    if id == None:
        numberlist.append(None)
    else: 
        numberlist.append(id.group(0))

    processtype = re.search('(?<=[0-9\_]\_)[a-z A-Z]+(?=\_)', title)
    if processtype != None:
        typelist.append(processtype.group(0))
    else:
        typelist.append("Onbekend")
        
    date = re.search("[0-9-?]+(?=.pdf$)", title)
    if date == None:
        datelist.append(None)
    else: 
        try: datelist.append(datetime.strptime(date.group(0), '%d-%m-%Y'))
        except: 
            try: datelist.append(datetime.strptime(date.group(0), '%-d-%-m-%Y'))
            except: datelist.append(date.group(0))

process["id"] = numberlist
process["type"] = typelist
process["date"] = datelist

# Remove uninteresting files

In [6]:
process.abstract = process.abstract.apply(lambda x: str(x))

for index, row in process.iterrows():
    if re.search(r'\bposter\b', process.loc[index].title): # Remove poster documents as they most likely do not contain useful info
        process.drop([index], axis = 0, inplace = True)
    elif re.search(r'checksum', process.loc[index].abstract): # Remove any file that still contains the word checksum
        process.drop([index], axis = 0, inplace = True)
process.abstract = process.abstract.apply(lambda x: re.sub(r'S?s?ubject', 'Onderwerp', x)) # Change the word "subject" to "onderwerp" for every file to make the data cleaning work for every file.

## Improve date column

In [7]:
#Convert normal date column to column with type datetime
def tryconvertdate(date):
    try:
        return pd.to_datetime(date, infer_datetime_format=True)
    except:
        pass

actualdatelist = process.date.apply(tryconvertdate)
actualdatelist = actualdatelist.tolist()
for i in range(1, len(process)):
    if pd.isnull(actualdatelist[i]):
        actualdatelist[i] = process.date.iloc[i]

In [9]:
#Handle edge cases so the entire column becomes of date time format
for i in range(1, len(actualdatelist)):
    if not isinstance(actualdatelist[i], datetime.datetime):
        if actualdatelist[i] == None:
            actualdatelist[i] = pd.Timestamp(None)
        elif not re.search("([?])", actualdatelist[i]) == None:## check if entry contains question marks
            actualdatelist[i] = pd.to_datetime(actualdatelist[i], errors = 'coerce')
        elif not re.search("[0-9]{4}[-][0-9]{4}", actualdatelist[i]) == None:
            actualdatelist[i] = pd.to_datetime(actualdatelist[i][0:4])
process["date"] = actualdatelist    

In [10]:
#Make types uniform
for val in process.index.values:
    if process.type.loc[val] == 'correspondentie':
        process.loc[val] = 'Correspondentie'

# Clean Data

In [11]:
# Clean the title
def cleanTitle(title):
    title = re.sub('^[0-9\.]+_+[a-z A-Z]+_', '', title)
    title = re.sub('[0-9\-]+.pdf$', '', title)
    title = re.sub('.msg_', ' ', title)
    return title

process.title = process.title.apply(cleanTitle)



In [12]:
# Remove email-adresses and links from abstract
process.abstract = process.abstract.apply(lambda x: re.sub(r'\S*@\S*\s?', '', x))
process.abstract = process.abstract.apply(lambda x: re.sub(r'(http|www)\S+', '', x))

# Properly split documents and extract texts

In [13]:
def splitdocuments(category):
    ''' Function that cleans and splits documents based on their contents.
    This Method does two things. First of all it tries to extract only the relevant text from a document.
    If the function notices a document consists of more then one document, it will try to split the document while adding
    them to the dataframe and removing the old one.
    '''
    for index, row in category.iterrows():                                                                                                          # We iterate over all rows in the input 
        occurences = [(m.start(0), m.end(0)) for m in re.finditer(r'\bOnderwerp\:? ?\b',row.abstract)]                                              # Find all documents in the entire document by scanning the amount of times 'Onderwerp' is mentioned    
        if len(occurences) > 1:                                                                                                                     # Enter if more than one document in the document was found
            maillist = []                                                                                                                           # Create an empty list to store emails in if 'Onderwerp' is found more than once
            rowdf = pd.DataFrame(category[0:0])
            full_mail = ''# Create empty dataframe with correct column names to store emails in temporarily
            for i in range(0,len(occurences)):                                                                                                      # For every document found within the document
                rowdf = rowdf.append(row)                                                                                                           # Since only the abstracts of each row need to be changed according to the amount of documents found within a document, each time the iteration is entered a new copy of the row is added to the datafram
               # Extract the single documents based on indices. By using regex, each document is scanned for the end(by using either Groet or Hoogachend). Then depending on which is found earlier. The starting index of that word is used.
                index1, index2 = None,None                                                                                                          # Create empty variables to store indexes in
                mail = row.abstract[occurences[i][1]:]                                                                                              # Find beginning of document
                if re.search(r'\bg?G?roete?n?\:? ?\b', mail):                                                                                       # Find possible end of document
                    index1 = re.search(r'\bg?G?roete?n?\:? ?\b', mail).span() 
                if re.search(r'\b.?h?H?oogachtend\:? ?,?\b', mail):                                                                                 # Find possible end of document
                    index2 = re.search(r'\b.?h?H?oogachtend\:? ?,?\b', mail).span()

                if index1 is not None and index2 is not None and index1[0] < index2[0]:                                                             # If both ends were found in a single document, get the end with the lowest beginning index value
                    maillist.append(mail[:index1[1]])
                elif index1 is not None and index2 is not None and index1[0] > index2[0]:
                    maillist.append(mail[:index2[1]])
                elif index1 is not None:
                    maillist.append(mail[:index1[1]])
                elif index2 is not None:
                    maillist.append(mail[:index2[1]])
                else:                                                                                                                               # If none of the identifiers were found, simply append the document
                    maillist.append(mail)
            if maillist:                                                                                                                            # Append all seperate changed abstract to a new list which then is added to the main inputted document. The original document is also removed.
                #category = category[category['id'] != row.id]
                for j in range(0, len(maillist)):
                    full_mail += maillist[j]
                #full_mail = rowdf.iloc[j,1]
                category.loc[index, 'abstract'] = full_mail 
                #category = pd.concat([category, rowdf])
        else:                                                                                                                                       # Only reached if document only consists of a single document, or if the document contains some edge case that was not defined. Does the rest as the for loop above.
            index5, index6, final_abstract = None, None, None
            temp_abstract = row.abstract
            if re.search(r'\bOnderwerp\:? ?\b', temp_abstract):
                temp_abstract = temp_abstract[re.search(r'\bOnderwerp\:? ?\b', temp_abstract).span()[1]:]     
            if re.search(r'\bg?G?roete?n?\:? ?\b', temp_abstract):
                index5 = re.search(r'\bg?G?roete?n?\:? ?\b', temp_abstract).span()
            if re.search(r'\b.?h?H?oogachtend,?\b', temp_abstract):
                index6 = re.search(r'\b.?h?H?oogachtend,?\b', temp_abstract).span()
                
            if index5 is not None and index6 is not None and index5[0] < index6[0]:
                final_abstract = temp_abstract[:index5[1]]
            if index5 is not None and index6 is not None and index5[0] > index6[0]:
                final_abstract = temp_abstract[:index6[1]]
            elif index5 is not None:
                final_abstract = temp_abstract[:index5[1]]
            elif index6 is not None:
                final_abstract = temp_abstract[:index6[1]]
            
            if not re.search(r'\bOnderwerp\:? ?\b', row.abstract):                                                                                 # If none of the identifiers are found simply append the document
                final_abstract = temp_abstract
            if final_abstract is not None:
                category.loc[index, 'abstract'] = final_abstract  
            
    return category
    

## Clean subsets

In [21]:
# Apply the previously created function to the relevant subsets
correspondentie = process[process['type'] == 'Correspondentie'].copy()
edited_correspondentie = splitdocuments(correspondentie)

mail = process[process['type'] == 'Mail'].copy()
edited_mail = splitdocuments(mail)

document = process[process['type'] == 'Document'].copy()
document.drop([1925], axis=0, inplace=True) # Column dropped due to very long abstract
for val in document.index.values:
    if re.search(r'Offerte', document.abstract.loc[val]): # Offertes are not interesting for text analysis so the abstracts containing this term are dropped
        document.drop([val], axis=0, inplace=True)

edited_document = splitdocuments(document)

# Concatenate and extract

In [67]:
# Concatenate the previously created subsets and extract them so that they can easily be loaded in into another python script
finaldf = pd.concat([edited_correspondentie, edited_mail, edited_document])
finaldf['abstract_length' ] = finaldf.abstract.apply(lambda x: len(x))
finaldf.abstract = finaldf.abstract.apply(lambda x: re.sub(r'\d+', "",x)) #remove digits from abstract
finaldf.abstract =  finaldf.abstract.apply(lambda x: re.sub(r'(?:^| )\w(?:$| )', ' ', x).strip()) #remove single letters surrounded by spaces
finaldf = finaldf.reset_index()
finaldf[['id','type','title','abstract', 'abstract_length']].to_excel(r'preprocessed_final_nosplit.xlsx')